In [1]:
import polars as pl

In [19]:
df = (
    pl.scan_csv("/home/gwatk/Downloads/pubchem.chembl.dataset4publication_inchi_smiles.tsv", separator="\t")
    .select(["DB", "Original_Assay_ID", "Gene_Symbol", "InChI", "SMILES"])
    .filter(pl.col("Gene_Symbol").is_in(["BRCA1", "HIF1A", "JUN", "STAT3", "TP53", "HSPA5"]))
    .collect()
)

In [20]:
df

DB,Original_Assay_ID,Gene_Symbol,InChI,SMILES
str,i64,str,str,str
"""chembl20""",845196,"""BRCA1""","""InChI=1/C18H14…","""O=C(NC=1C=C2N=…"
"""chembl20""",737623,"""TP53""","""InChI=1/C18H14…","""O=C(NC=1C=C2N=…"
"""pubchem""",624202,"""BRCA1""","""InChI=1/C18H14…","""O=C(NC1=CC=2NC…"
"""pubchem_screen…",871,"""STAT3""","""InChI=1/C18H14…","""O=C(NC1=CC=2NC…"
"""pubchem""",624202,"""BRCA1""","""InChI=1/C21H21…","""S(C=1N(C(=NN1)…"
"""pubchem_screen…",871,"""STAT3""","""InChI=1/C21H21…","""S(C=1N(C(=NN1)…"
"""pubchem""",504706,"""TP53""","""InChI=1/C21H21…","""S(C=1N(C(=NN1)…"
"""pubchem""",624202,"""BRCA1""","""InChI=1/C20H16…","""S(CC1=CC=C(F)C…"
"""pubchem""",504706,"""TP53""","""InChI=1/C20H16…","""S(CC1=CC=C(F)C…"


In [28]:
q = (
    df.lazy()
    .groupby("Gene_Symbol")
    .agg(
        pl.count(),
        pl.col("Original_Assay_ID").n_unique().alias("n_assays"),
        pl.col("InChI").n_unique().alias("n_inchi"),
        pl.col("DB").n_unique().alias("n_db"),
    )
)

In [29]:
q.collect()

Gene_Symbol,count,n_assays,n_inchi,n_db
str,u32,u32,u32,u32
"""JUN""",3387,18,3387,2
"""STAT3""",209395,90,209378,3
"""HSPA5""",3154,4,3154,1
"""TP53""",388890,25,388867,3
"""HIF1A""",10303,37,10300,3
"""BRCA1""",372531,8,372525,2


In [6]:
qq = (
    pl.scan_csv("/home/gwatk/Downloads/pubchem.chembl.dataset4publication_inchi_smiles.tsv", separator="\t")
    .select(["DB", "Original_Assay_ID", "Gene_Symbol", "InChI", "SMILES"])
    .groupby("Gene_Symbol")
    .agg(
        pl.count(),
        pl.col("Original_Assay_ID").n_unique().alias("n_assays"),
        pl.col("InChI").n_unique().alias("n_inchi"),
        pl.col("DB").n_unique().alias("n_db"),
    )
    .take_every(20)
    .fetch(50000)
)

In [10]:
qq.head(25)

Gene_Symbol,count,n_assays,n_inchi,n_db
str,u32,u32,u32,u32
"""PTPN7""",84,1,84,1
"""CA13""",2,1,2,1
"""LOC400927-CSNK…",1,1,1,1
"""GALK1""",185,2,185,1
"""GRIK1""",1,1,1,1
"""RAC1""",137,2,137,2
"""CCKBR""",2,2,2,2
"""KDM4C""",67,1,67,1
"""CASP2""",1,1,1,1
